In [1]:
import os
os.chdir('..')

In [21]:
import numpy as np
import pandas as pd
import scipy as sp
import plotly.express as px
from plotly.offline import init_notebook_mode

In [88]:
from pymarkowitz import *

In [23]:
init_notebook_mode(connected=True)

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Reading Data
- 1990 - 2000 Selected SP500 Component Data

In [26]:
sp500 = pd.read_csv("datasets/sp500_1990_2000.csv", index_col='DATE').drop(["Unnamed: 0"], axis=1)
sp500.head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/02/1990,1.330,5.875,3.874,1.282,8.4918,5.0313,33.125,1.1750,148.9705,6.1875,...,10.6667,2.8281,32.250,4.9662,5.9219,11.6129,12.5000,1.0417,25.4128,3.5000
01/03/1990,1.339,5.813,3.888,1.345,8.5382,5.1563,33.000,1.1500,148.7940,6.1563,...,10.4583,2.7031,32.375,5.0755,5.8906,11.3571,12.3750,1.0417,25.5775,3.5625
01/04/1990,1.344,5.625,3.881,1.408,8.4918,5.0313,32.500,1.1250,146.6759,6.1563,...,10.4167,2.6719,32.750,4.9194,5.8594,11.2548,12.2500,1.0000,25.6324,3.5625
01/05/1990,1.348,5.750,3.839,1.440,8.0741,5.0313,31.750,1.1250,142.2633,6.1563,...,10.2500,2.5938,32.000,4.9350,5.7969,11.2036,12.2188,1.0625,25.6873,3.5625
01/08/1990,1.357,5.750,3.839,1.456,8.2134,4.9844,31.625,1.1333,141.3807,6.1563,...,10.1667,2.6719,32.250,4.9974,5.8750,11.2036,12.3750,1.0208,25.5226,3.5625


In [31]:
### Sample Plot Series
px.line(sp500.reset_index(), x='DATE', y='MSFT')

### ReturnGenerator
- Daily Return
- Rolling Return
- Collapsed Return
- Mean Return

** In most cases, daily returns are used to construct covariance matrices and mean returns are passed in as expected return in markowitz optimization problems

In [78]:
### Initialize a Return Calculator
ret_generator = ReturnGenerator(sp500)

#### Daily Return Methods
- Discrete DoD Return
- Continuous Return (Set log=True)

** Calling ret_generator.calc_return(method='daily') yields the identical result as sp500.pct_change().dropna(how='any')

In [79]:
### Daily Return
ret_generator.calc_return(method='daily').head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/03/1990,0.006767,-0.010553,0.003614,0.049142,0.005464,0.024844,-0.003774,-0.021277,-0.001185,-0.005042,...,-0.019537,-0.044199,0.003876,0.022009,-0.005285,-0.022027,-0.010000,0.000000,0.006481,0.017857
01/04/1990,0.003734,-0.032341,-0.001800,0.046840,-0.005434,-0.024242,-0.015152,-0.021739,-0.014235,0.000000,...,-0.003978,-0.011542,0.011583,-0.030756,-0.005297,-0.009008,-0.010101,-0.040031,0.002146,0.000000
01/05/1990,0.002976,0.022222,-0.010822,0.022727,-0.049189,0.000000,-0.023077,0.000000,-0.030084,0.000000,...,-0.016003,-0.029230,-0.022901,0.003171,-0.010667,-0.004549,-0.002547,0.062500,0.002142,0.000000
01/08/1990,0.006677,0.000000,0.000000,0.011111,0.017253,-0.009322,-0.003937,0.007378,-0.006204,0.000000,...,-0.008127,0.030110,0.007812,0.012644,0.013473,0.000000,0.012784,-0.039247,-0.006412,0.000000
01/09/1990,-0.009580,-0.021739,-0.003621,-0.021978,0.000000,-0.003130,-0.007905,0.000000,-0.006242,-0.010152,...,-0.004102,-0.029230,0.000000,-0.018750,-0.026587,-0.018262,-0.012622,0.081701,-0.015053,0.000000


In [80]:
### Continuous Return
ret_generator.calc_return(method='daily', log=True).head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/03/1990,0.006744,-0.010609,0.003607,0.047973,0.005449,0.024541,-0.003781,-0.021506,-0.001186,-0.005055,...,-0.019731,-0.045206,0.003868,0.021770,-0.005299,-0.022273,-0.010050,0.000000,0.006460,0.0177
01/04/1990,0.003727,-0.032876,-0.001802,0.045776,-0.005449,-0.024541,-0.015267,-0.021979,-0.014337,0.000000,...,-0.003986,-0.011609,0.011516,-0.031238,-0.005311,-0.009048,-0.010152,-0.040854,0.002144,0.0000
01/05/1990,0.002972,0.021979,-0.010881,0.022473,-0.050440,0.000000,-0.023347,0.000000,-0.030546,0.000000,...,-0.016133,-0.029666,-0.023167,0.003166,-0.010724,-0.004560,-0.002550,0.060625,0.002140,0.0000
01/08/1990,0.006654,0.000000,0.000000,0.011050,0.017106,-0.009365,-0.003945,0.007351,-0.006223,0.000000,...,-0.008160,0.029666,0.007782,0.012565,0.013383,0.000000,0.012703,-0.040038,-0.006432,0.0000
01/09/1990,-0.009626,-0.021979,-0.003627,-0.022223,0.000000,-0.003135,-0.007937,0.000000,-0.006262,-0.010204,...,-0.004110,-0.029666,0.000000,-0.018928,-0.026947,-0.018431,-0.012703,0.078534,-0.015168,0.0000


#### Rolling Return Methods
- Set window=x in calc_return
- Will automatically skip the first x days without rolling data available


** Calling ret_generator.calc_return(method='daily') yields the identical result as sp500.pct_change(x).dropna(how='any')

In [81]:
### Rolling Monthly Return
ret_generator.calc_return(method='rolling', window=30).head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
02/13/1990,-0.073684,-0.180766,-0.072199,0.326833,-0.109294,0.114901,-0.079245,-0.106383,-0.105450,-0.101010,...,-0.089850,-0.160214,-0.038760,-0.157223,-0.092352,-0.163000,-0.055000,-0.040031,-0.075592,-0.053571
02/14/1990,-0.086632,-0.204370,-0.066538,0.270632,-0.114134,0.072727,-0.079545,-0.086957,-0.100831,-0.096454,...,-0.067726,-0.121379,-0.038610,-0.169225,-0.084881,-0.135140,-0.053026,-0.040031,-0.111586,-0.070175
02/15/1990,-0.090030,-0.144356,-0.052255,0.281250,-0.120222,0.130424,-0.065385,-0.059289,-0.068592,-0.065997,...,-0.064003,-0.076013,-0.026718,-0.152397,-0.072004,-0.113640,-0.012751,0.083300,-0.147751,-0.070175
02/16/1990,-0.106083,-0.173913,-0.047356,0.241667,-0.086202,0.099378,-0.047244,-0.074044,-0.047146,-0.055845,...,-0.048780,-0.048192,-0.027344,-0.170881,-0.075471,-0.114151,-0.017907,0.009788,-0.141027,-0.043846
02/19/1990,-0.112012,-0.173913,-0.047356,0.228022,-0.101700,0.109722,-0.043478,-0.080826,-0.041198,-0.055845,...,-0.040987,-0.076013,-0.034884,-0.181234,-0.087762,-0.114151,-0.030303,0.051038,-0.135484,-0.043846


#### Collapsed Return Methods
- Set window=x in calc_return
- Will calculate return, from the first available observation, separated by window

In [82]:
### Annual Return
ret_generator.calc_return(method='collapse', window=261).head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/02/1991,0.168421,-0.489362,0.249097,0.419657,0.027050,0.260867,-0.162264,0.070894,-0.115818,-0.085851,...,-0.011719,-0.099431,-0.282946,-0.342757,0.266485,-0.259909,0.015000,-0.020063,-0.373650,0.080371
01/02/1992,0.361647,1.708333,0.554929,1.356044,0.495809,-0.270942,0.225225,0.576174,0.296482,-0.011050,...,0.237153,0.791393,0.675676,0.435386,0.970840,0.303563,0.182266,1.693966,0.868966,0.388411
01/01/1993,0.008507,-0.338462,-0.096660,-0.534981,-0.127061,0.250011,-0.025735,0.159683,0.198966,0.262559,...,0.015972,0.181501,0.151613,0.046659,0.076110,0.347031,0.016667,1.954545,0.169744,0.809524
01/03/1994,-0.500000,-0.302326,-0.049389,0.349047,-0.088680,-0.032432,0.128302,-0.008130,0.129850,0.283186,...,0.033019,0.107261,0.467787,0.222934,-0.198431,0.193220,0.045082,-0.102560,0.111987,-0.039474
01/03/1995,0.284911,-0.333333,0.134214,0.391078,0.386686,0.670385,-0.120401,0.168939,0.124463,-0.131034,...,-0.050228,-0.026186,-0.230916,0.052094,-0.186275,-0.116473,-0.047059,-0.274284,0.100709,-0.006849


In [83]:
ret_generator.calc_return(method='collapse', window=261, log=True).head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/02/1991,0.155653,-0.672094,0.222421,0.350415,0.026690,0.231800,-0.177052,0.068493,-0.123092,-0.089761,...,-0.011788,-0.104728,-0.332604,-0.419702,0.236246,-0.300982,0.014889,-0.020267,-0.467846,0.077305
01/02/1992,0.308695,0.996333,0.441430,0.856984,0.402667,-0.316002,0.203125,0.455001,0.259655,-0.011111,...,0.212813,0.582994,0.516216,0.361434,0.678460,0.265101,0.167433,0.991014,0.625386,0.328160
01/01/1993,0.008471,-0.413187,-0.101656,-0.765678,-0.135890,0.223152,-0.026072,0.148147,0.181459,0.233140,...,0.015846,0.166786,0.141163,0.045603,0.073352,0.297903,0.016529,1.083345,0.156785,0.593064
01/03/1994,-0.693147,-0.360003,-0.050651,0.299399,-0.092861,-0.032970,0.120714,-0.008164,0.122085,0.249346,...,0.032485,0.101889,0.383756,0.201253,-0.221184,0.176656,0.044095,-0.108209,0.106149,-0.040274
01/03/1995,0.250689,-0.405465,0.125940,0.330079,0.326916,0.513054,-0.128290,0.156096,0.117306,-0.140452,...,-0.051534,-0.026535,-0.262555,0.050782,-0.206132,-0.123833,-0.048202,-0.320597,0.095955,-0.006873


### Mean Historical Return

#### Arithmetic
- Calculates with arithmetic mean of all observations

#### Geometric
- Calculates with geometric mean between the first observation and the last observation

** Time Scale to Annual/Monthly Return

In [84]:
### Daily Arithmetic Mean Return
ret_generator.calc_mean_return(method='arithmetic', time_scaling=1)

AAPL    0.000523
ABMD    0.001916
ABT     0.000757
ADBE    0.001763
ADM     0.000342
          ...   
WY      0.000396
XOM     0.000530
XRAY    0.001144
XRX     0.000036
ZION    0.001213
Length: 203, dtype: float64

In [85]:
### Daily Geometric Mean Return
ret_generator.calc_mean_return(method='geometric', time_scaling=1)

AAPL   -0.000078
ABMD    0.000494
ABT     0.000600
ADBE    0.001089
ADM     0.000181
          ...   
WY      0.000203
XOM     0.000435
XRAY    0.000881
XRX    -0.000256
ZION    0.001005
Length: 203, dtype: float64

In [86]:
### Annual Arithmetic Mean Return
ret_generator.calc_mean_return(method='arithmetic', time_scaling=252)

AAPL    0.131691
ABMD    0.482786
ABT     0.190670
ADBE    0.444226
ADM     0.086138
          ...   
WY      0.099815
XOM     0.133441
XRAY    0.288408
XRX     0.009090
ZION    0.305800
Length: 203, dtype: float64

In [87]:
### Annual Geometric Mean Return
ret_generator.calc_mean_return(method='geometric', time_scaling=252)

AAPL   -0.019490
ABMD    0.132611
ABT     0.163290
ADBE    0.315520
ADM     0.046748
          ...   
WY      0.052394
XOM     0.115687
XRAY    0.248554
XRX    -0.062523
ZION    0.287998
Length: 203, dtype: float64